In [1]:
# Import statements
import numpy as np 
import pandas as pd
import yfinance as yf
import requests, sys
from bs4 import BeautifulSoup
# from xbrl import XBRLParser, GAAP, GAAPSerializer
import urllib

In [2]:
'''
Method that extracts SPX data from the Wikipedia page on the list of S&P 500 companies
    @param: None
    @return: a Pandas dataframe with columns for the (1) ticker, (2) security, 
    (3) sector and (4) CIK ID [to search SEC Edgar]
'''
def getSPXTickers():
    URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    res = requests.get(URL).text
    soup = BeautifulSoup(res, 'lxml')
    
    # Generates a CIK set to only add unique companies with a unique CIK ID
    # This is important since several companies might have different share classes
    CIKS = set()
    
    # Sets up the SPX dictionary to store the values from each company
    spx_dict = {}
    spx_dict['ticker'] = [] ; spx_dict['security'] = [] 
    spx_dict['sector'] = [] ; spx_dict['cik'] = []
    
    # Loops through the first 506 (505) table rows on the Wikipedia page
    # 5 companies have different share classes available to investors
    for items in soup.find('table', class_='wikitable').find_all('tr')[1:506]:
        data = items.find_all(['td'])
        if data[7] not in CIKS:
            CIKS.add(data[7])
            
            # This little bit of trickery accounts for the edge-case of the security 'BF.B' 
            # which should be 'BF-B'
            if '.' in data[0].text:
                spx_dict['ticker'].append(data[0].text.strip('\n').replace('.','-'))
            else:
                spx_dict['ticker'].append(data[0].text.strip('\n'))
                
            spx_dict['security'].append(data[1].text.strip('\n'))
            spx_dict['sector'].append(data[3].text.strip('\n'))
            spx_dict['cik'].append(data[7].text.strip('\n'))
    return pd.DataFrame.from_dict(spx_dict)

# Code to execute below:
spx_df = getSPXTickers()

In [11]:
# Note for this method: 'institutional holders error' comes from a try-except in Line 283 of yfinance base.py
# This try-except was added 

'''
Method that stores information on each SPX stock for further processing
    @param: a Pandas Core Series with each ticker
    @param: a list of each field to be extracted from the 'security.info' dictionary
    @return: a Pandas dataframe with columns for the (1) ticker, and (2-X) for each extracted
    field from the 2nd param
'''
def storeSPXParams(tickers, fields):
    spxParams_dict = {}
    spxParams_dict['ticker'] = []
    for field in fields:
        spxParams_dict[field] = []
    
    # Loops through all tickers and populates the spxParams dictionary
    for ticker in tickers:
        yf_ticker = yf.Ticker(ticker)
        spxParams_dict['ticker'].append(ticker)
        for field in fields:
            try:
                spxParams_dict[field].append(yf_ticker.info[field])
            except:
                spxParams_dict[field].append(-1)
                print("{} failed for {}".format(str(field), ticker))
    return pd.DataFrame.from_dict(spxParams_dict)

# Code to execute below:
fields = ['beta', 'trailingPE', 'marketCap', 'priceToSalesTrailing12Months', 
         'enterpriseToEbitda', 'bookValue', 'previousClose', 'sharesShort']
spx_params = storeSPXParams(spx_df.ticker,fields)

ticker                                 ALB
beta                               1.46429
trailingPE                         22.4302
marketCap                       9826415616
priceToSalesTrailing12Months       2.91135
enterpriseToEbitda                  14.636
bookValue                           37.846
previousClose                         91.6
sharesShort                       12186827
Name: 15, dtype: object

In [116]:
'''
Method to populate a dictionary 
'''
def get10KEdgarParams(CIK, params, start_year=2012, end_year=2019):
    # Step 1: Navigate to the SEC search results page for the given company's 10-K filings
    URL = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-K&dateb=&owner=exclude&count=100".format(str(CIK))
    res = requests.get(URL).text
    soup = BeautifulSoup(res, 'lxml')
    step1rows = soup.find('table', class_='tableFile2').find_all('tr')[:100]
    
    # Loops through until we reach the end year - we descend from the end year ot the start year
    index = 1
    while (step1rows[index].find_all('td')[3].text[:4] != str(end_year + 1) or step1rows[index].find_all('td')[0].text != '10-K'):
        index += 1
        
    # Step 2: For each year from the end year to the start year, relevant params are extracted and populated
    
    # Populates dataframe (columns = parameters, index = years)
    years = range(start_year, end_year + 1)
    params_df = pd.DataFrame(columns=params, index=years)
    cur_year = end_year # this is an int
    
    while (step1rows[index].find_all('td')[3].text[:4] != str(start_year)):
        if index > (len(step1rows) - 1):
            break
        if step1rows[index].find_all('td')[0].text != '10-K':
            index += 1
            pass
        else:
            print(step1rows[index].find_all('td')[3].text[:4])
            url_ext = step1rows[index].find_all('td')[1].find_all('a')[0]['href']
            URL2 = "https://www.sec.gov{}".format(str(url_ext))
            print(URL2)
            res = requests.get(URL2).text
            soup = BeautifulSoup(res, 'html.parser')
            
            try:
                table_tags = soup.find('table', class_='tableFile', summary='Data Files')
            except:
                print("XML XBRL Tables not found")
                break
                
            # Step 3: Parses through the different table tags to find the XBRL Instance XML Document
            table_cells = table_tags.find_all('td')
            for i in range(len(table_cells)):
                if 'INS' in table_cells[i].text:
                    try:
                        doc_ext = table_cells[i+1].a['href']
                        break
                    except:
                        print("Alternative table format")
                    try:
                        doc_ext = table_cells[i-1].a['href']
                        break
                    except:
                        print("No table format found")
                        break
                        
            # Step 4: Generates the URL for the 10-K file
            URL3 = "https://www.sec.gov{}".format(str(doc_ext))
            res = requests.get(URL3).text
            soup = BeautifulSoup(res, 'lxml')
            
            # Step 4: Loops through all the parameters to find them in the 10-K XML File
            searchable_file = soup.find('html')
            for param in params:
                try:
#                     for v in searchable_file.find_all(param):
#                         print(v['contextref'])
                    params_df[param][cur_year] = searchable_file.find(param).text #['contextref']
                except:
                    print("Parameter {} not found".format(str(param)))
            
            # Step 5: Update relevant parameters before looping
            cur_year -= 1
            index += 1
    
    return params_df      

In [30]:
for i in range(50,60):
    cik = spx_df.cik[i]
    URL = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-K&dateb=&owner=exclude&count=100".format(str(cik))
    res = requests.get(URL).text
    soup = BeautifulSoup(res, 'lxml')
    #     print(URL)
    x = soup.find('table', class_='tableFile2').find_all('tr')[:100]
    url_ext = x[1].find_all('td')[1].find_all('a')[0]['href']
    # print(url_ext)
    URL2 = "https://www.sec.gov{}".format(str(url_ext))
    res = requests.get(URL2).text
    soup = BeautifulSoup(res, 'html.parser')
    table_tags = soup.find('table', class_='tableFile', summary='Data Files')
    next_bool = False 
    table_cells = table_tags.find_all('td')
    for i in range(len(table_cells)):
        if 'INS' in table_cells[i].text:
            try:
                doc_ext = table_cells[i+1].a['href']
                break
            except:
                print("Alternative table format")
            try:
                doc_ext = table_cells[i-1].a['href']
                break
            except:
                print("No table format found")
                break
    # print(doc_ext)
    URL3 = "https://www.sec.gov{}".format(doc_ext)
    res = requests.get(URL3).text
    soup = BeautifulSoup(res, 'lxml')
    searchable = soup.find('html')
    x = searchable.find_all('us-gaap:revenues')
    if not x:
        x = searchable.find_all('us-gaap:revenuefromcontractwithcustomerexcludingassessedtax')
    if not x:
        x = searchable.find_all('us-gaap:regulatedandunregulatedoperatingrevenue')
    print(x[0])

<us-gaap:revenuefromcontractwithcustomerexcludingassessedtax contextref="FD2019Q4YTD_srt_ProductOrServiceAxis_us-gaap_ProductMember" decimals="-3" id="d73780586e950-wk-Fact-EC9892A4D0F9B78AFD46CC7273422894" unitref="usd">2021150000</us-gaap:revenuefromcontractwithcustomerexcludingassessedtax>
<us-gaap:revenues contextref="P01_01_2019To03_31_2019" decimals="-5" unitref="Unit_USD">1990600000</us-gaap:revenues>
<us-gaap:revenues contextref="FD2019Q4YTD" decimals="-5" id="d52406991e1444-wk-Fact-01F249567922690948FBCD15C8AABD00" unitref="usd">10086800000</us-gaap:revenues>
<us-gaap:revenues contextref="FROM_Jan01_2019_TO_Dec31_2019_Entity_0000732717_srt_ProductOrServiceAxis_t_ServiceAndOtherMember" decimals="-6" id="ID_2201" unitref="USD">163499000000</us-gaap:revenues>
<us-gaap:revenues contextref="FD2019Q4YTD_srt_ConsolidationItemsAxis_us-gaap_OperatingSegmentsMember_us-gaap_StatementBusinessSegmentsAxis_ato_DistributionSegmentMember" decimals="-3" id="d8306332e928-wk-Fact-58AE71FF44AD334

In [172]:
spx_df.cik[61]

'0000009389'

In [22]:
URL2

'https://www.sec.gov/Archives/edgar/data/1065088/000120677420001423/0001206774-20-001423-index.htm'

In [117]:
# ans = get10KEdgarParams(spx_df.cik[34], ['us-gaap:netincomelossavailabletocommonstockholdersbasic'], start_year=2012, end_year=2019)
ans = get10KEdgarParams(spx_df.cik[24], ['us-gaap:netincomeloss'], start_year=2012, end_year=2019)
# ans = get10KEdgarParams(spx_df.cik[55], ['us-gaap:profitloss'], start_year=2012, end_year=2019)

2020
https://www.sec.gov/Archives/edgar/data/764180/000076418020000018/0000764180-20-000018-index.htm
2019
https://www.sec.gov/Archives/edgar/data/764180/000076418019000023/0000764180-19-000023-index.htm
2018
https://www.sec.gov/Archives/edgar/data/764180/000076418018000028/0000764180-18-000028-index.htm
2017
https://www.sec.gov/Archives/edgar/data/764180/000076418017000028/0000764180-17-000028-index.htm
2016
https://www.sec.gov/Archives/edgar/data/764180/000076418016000128/0000764180-16-000128-index.htm
2015
https://www.sec.gov/Archives/edgar/data/764180/000076418015000022/0000764180-15-000022-index.htm
2014
https://www.sec.gov/Archives/edgar/data/764180/000076418014000025/0000764180-14-000025-index.htm
2013
https://www.sec.gov/Archives/edgar/data/764180/000076418013000024/0000764180-13-000024-index.htm
Parameter us-gaap:netincomeloss not found


In [118]:
ans

,us-gaap:netincomeloss
2012,NaN
2013,3390000000
2014,4180000000
2015,4535000000
2016,5070000000
2017,5241000000
2018,14239000000
2019,-1293000000


In [12]:
spx_df.cik[162]

'0001065088'